In [13]:
import torch
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from torch.utils.data import DataLoader, TensorDataset
import json
import pdb
import logging.handlers
import argparse
import os
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from networkx import DiGraph, relabel_nodes, all_pairs_shortest_path_length
from sklearn_hierarchical_classification.constants import ROOT
from sklearn_hierarchical_classification.metrics import h_fbeta_score, h_recall_score, h_precision_score, fill_ancestors, multi_labeled
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from tqdm import tqdm
from ordered_set import OrderedSet

In [14]:
import json

# 文件路径
file_paths = ['train.json', 'validation.json','dev_subtask1_en.json']

# 读取数据
merged_data = []

for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as file:  # 指定文件编码
        data = json.load(file)
        merged_data.extend(data)

# 写入新的 JSON 文件
with open('merged_file.json', 'w', encoding='utf-8') as merged_file:  # 指定写入文件编码为 UTF-8
    json.dump(merged_data, merged_file, indent=2, ensure_ascii=False)


In [15]:
file_path = 'merged_file.json'
# Read the JSON file into a list of dictionaries
with open(file_path, 'r', encoding='utf-8') as file:
    data_train = json.load(file)

rows = []
possible_labels = OrderedSet(label for entry in data_train for label in entry['labels'])

for entry in data_train:
    id = entry['id']
    text = entry['text']
    labels = entry['labels']
    row = {}
    row['id'] = id
    row['text'] = text

    for label in possible_labels:
        row[label] = int(label in labels)

    rows.append(row)

df = pd.DataFrame(rows)
data_train = df

# Check and replace NaN values
for i in data_train.index:
    if pd.isna(data_train["text"][i]):
        data_train["text"][i] = ""




In [16]:
file_path = 'en_subtask1_test_unlabeled.json'
# Read the JSON file into a list of dictionaries
with open(file_path, 'r', encoding='utf-8') as file:
    data_validation = json.load(file)

rows = []

for entry in data_validation:
    id = entry['id']
    text = entry['text']
    row = {}
    row['id'] = id
    row['text'] = text


    rows.append(row)

df = pd.DataFrame(rows)
data_validation = df

# Check and replace NaN values
for i in data_validation.index:
    if pd.isna(data_validation["text"][i]):
        data_validation["text"][i] = ""

In [17]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings = tokenizer(list(data_train["text"]), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(list(data_validation['text']), truncation=True, padding=True, return_tensors='pt')

In [18]:

class CustomDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = torch.tensor(labels.values, dtype=torch.float32) if labels is not None else None

    def __getitem__(self, idx):
        if self.labels is not None:
            return {key: val[idx].to(device) for key, val in self.encodings.items()}, self.labels[idx].to(device)
        else:
            return {key: val[idx].to(device) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.labels) if self.labels is not None else len(self.encodings["input_ids"])






In [19]:
labels = sorted([col for col in data_train.columns if col not in ["text", "id"]])

train_dataset = CustomDataset(train_encodings, data_train[labels])
test_dataset = CustomDataset(test_encodings)


train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(labels)).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.BCEWithLogitsLoss()



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
print(labels)

['Appeal to authority', 'Appeal to fear/prejudice', 'Bandwagon', 'Black-and-white Fallacy/Dictatorship', 'Causal Oversimplification', 'Doubt', 'Exaggeration/Minimisation', 'Flag-waving', 'Glittering generalities (Virtue)', 'Loaded Language', "Misrepresentation of Someone's Position (Straw Man)", 'Name calling/Labeling', 'Obfuscation, Intentional vagueness, Confusion', 'Presenting Irrelevant Data (Red Herring)', 'Reductio ad hitlerum', 'Repetition', 'Slogans', 'Smears', 'Thought-terminating cliché', 'Whataboutism']


In [21]:
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset, DataLoader
from networkx import DiGraph
from torch import nn

G = DiGraph()
G.add_edge(ROOT, "Logos")
G.add_edge("Logos", "Repetition")
G.add_edge("Logos", "Obfuscation, Intentional vagueness, Confusion")
G.add_edge("Logos", "Reasoning")
G.add_edge("Logos", "Justification")
G.add_edge('Justification', "Slogans")
G.add_edge('Justification', "Bandwagon")
G.add_edge('Justification', "Appeal to authority")
G.add_edge('Justification', "Flag-waving")
G.add_edge('Justification', "Appeal to fear/prejudice")
G.add_edge('Reasoning', "Simplification")
G.add_edge('Simplification', "Causal Oversimplification")
G.add_edge('Simplification', "Black-and-white Fallacy/Dictatorship")
G.add_edge('Simplification', "Thought-terminating cliché")
G.add_edge('Reasoning', "Distraction")
G.add_edge('Distraction', "Misrepresentation of Someone's Position (Straw Man)")
G.add_edge('Distraction', "Presenting Irrelevant Data (Red Herring)")
G.add_edge('Distraction', "Whataboutism")
G.add_edge(ROOT, "Ethos")
G.add_edge('Ethos', "Appeal to authority")
G.add_edge('Ethos', "Glittering generalities (Virtue)")
G.add_edge('Ethos', "Bandwagon")
G.add_edge('Ethos', "Ad Hominem")
G.add_edge('Ethos', "Transfer")
G.add_edge('Ad Hominem', "Doubt")
G.add_edge('Ad Hominem', "Name calling/Labeling")
G.add_edge('Ad Hominem', "Smears")
G.add_edge('Ad Hominem', "Reductio ad hitlerum")
G.add_edge('Ad Hominem', "Whataboutism")
G.add_edge(ROOT, "Pathos")
G.add_edge('Pathos', "Exaggeration/Minimisation")
G.add_edge('Pathos', "Loaded Language")
G.add_edge('Pathos', "Appeal to (Strong) Emotions")
G.add_edge('Pathos', "Appeal to fear/prejudice")
G.add_edge('Pathos', "Flag-waving")
G.add_edge('Pathos', "Transfer") 

In [22]:
class HierarchicalLoss(nn.Module):
    def __init__(self, alpha=1.0):
        super(HierarchicalLoss, self).__init__()
        self.alpha = alpha

    def forward(self, logits, labels, hierarchy):
        loss = nn.BCEWithLogitsLoss()(logits, labels)

        # Add hierarchical regularization
        for i in range(labels.size(1)):
            for j in range(labels.size(1)):
                if i != j and hierarchy.has_node(i) and hierarchy.has_node(j) and nx.has_path(hierarchy, source=i, target=j):
                    loss += self.alpha * torch.relu(logits[:, i] - logits[:, j])

        return loss

In [23]:
hierarchical_loss = HierarchicalLoss(alpha=0.1)
num_epochs = 1
model.to(device)

thresholds = {'Appeal to authority':0.25, 'Appeal to fear/prejudice':0.126, 'Bandwagon':0.072, 'Black-and-white Fallacy/Dictatorship':0.23, 
             'Causal Oversimplification':0.104, 'Doubt':0.129, 'Exaggeration/Minimisation':0.131, 'Flag-waving':0.18, 'Glittering generalities (Virtue)':0.16, 
             'Loaded Language':0.45, "Misrepresentation of Someone's Position (Straw Man)":0.064, 'Name calling/Labeling':0.39, 
             'Obfuscation, Intentional vagueness, Confusion':0.05, 'Presenting Irrelevant Data (Red Herring)':0.063, 
             'Reductio ad hitlerum':0.0642, 'Repetition':0.119, 'Slogans':0.2, 'Smears':0.5, 'Thought-terminating cliché':0.17, 'Whataboutism':0.108}
for epoch in tqdm(range(num_epochs)):
    model.train()
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        inputs, labels = batch
        outputs = model(**inputs).logits
        loss = hierarchical_loss(outputs, labels, G)
        loss.backward()
        optimizer.step()
    torch.save(model, "roberta_subtask1_epoch_" + str(epoch) + ".pt")



100%|██████████| 1/1 [13:51<00:00, 831.23s/it]


In [26]:
model.eval()
all_preds = []
all_labels = []
label_name = ['Appeal to authority', 'Appeal to fear/prejudice', 'Bandwagon', 'Black-and-white Fallacy/Dictatorship', 
             'Causal Oversimplification', 'Doubt', 'Exaggeration/Minimisation', 'Flag-waving', 'Glittering generalities (Virtue)', 
             'Loaded Language', "Misrepresentation of Someone's Position (Straw Man)", 'Name calling/Labeling', 
             'Obfuscation, Intentional vagueness, Confusion', 'Presenting Irrelevant Data (Red Herring)', 
             'Reductio ad hitlerum', 'Repetition', 'Slogans', 'Smears', 'Thought-terminating cliché','Whataboutism']
with torch.no_grad():
    for batch in test_loader:
        inputs = batch
        outputs = torch.sigmoid(model(**inputs).logits)
        binary_preds = (outputs.cpu().numpy() > np.array([thresholds[label] for label in label_name])).astype(int)
        all_preds.extend(binary_preds)



In [27]:
output_list = []

for i, entry in data_validation.iterrows():
    entry_id = entry['id']
    text = entry['text']
    predicted_labels = [label for label, pred in zip(possible_labels, all_preds[i]) if pred == 1]

    output_entry = {
        "id": entry_id,
        "text": text,
        "labels": predicted_labels
    }

    output_list.append(output_entry)

# Write the result to a JSON file
output_file_path = 'predictions.json'
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    json.dump(output_list, output_file, indent=2)
